In [ ]:
Goal: csv_1="D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\proteomic_mri_with_labels.csv"
It hs columns 'RID' 'VISCODE'
The goal is to match  the csv_1 with csv_2 below and join their columns for the same subject
csv_2="C:\Users\User\github_repos\AD_CN_all_available_data_final\AD_CN_all_available_data.csv"
It has columns Image Data ID	Subject	Group	Sex	Age	Visit	Modality	Description	Type	Acq Date	Format	Downloaded

The way to match csv_1 to csv_2 is to:

1)use the column 'RID' in csv_1 and the column 'Subject' in csv_2

To match RID to Subject, see examples below.

subject_id	RID
067_S_0029	29
023_S_0031	31
023_S_0061	61
136_S_0086	86
023_S_0139	139
127_S_0260	260

2) Match VISCODE in csv_1 to Visit in csv_2





<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\User\AppData\Local\Temp\ipykernel_19844\3930734027.py:1: SyntaxWarning: invalid escape sequence '\A'
  Goal: csv_1="D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\proteomic_mri_with_labels.csv"


SyntaxError: unmatched ')' (3930734027.py, line 9)

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

def extract_rid_from_subject(subject_id):
    """
    Extract RID from Subject ID format like '067_S_0029' -> 29
    
    Args:
        subject_id (str): Subject ID in format 'XXX_S_YYYY'
        
    Returns:
        int: RID number
    """
    if pd.isna(subject_id):
        return None
    
    # Extract the number after the last underscore
    match = re.search(r'_(\d+)$', str(subject_id))
    if match:
        return int(match.group(1))
    return None

def normalize_viscode(viscode):
    """
    Normalize VISCODE/Visit codes for matching
    
    Args:
        viscode (str): Visit code
        
    Returns:
        str: Normalized visit code
    """
    if pd.isna(viscode):
        return None
    
    viscode = str(viscode).lower().strip()
    
    # Common mappings
    visit_mappings = {
        'bl': 'bl',
        'baseline': 'bl',
        'm00': 'bl',
        'sc': 'sc',
        'screening': 'sc',
        'm06': 'm06',
        'm12': 'm12',
        'm18': 'm18',
        'm24': 'm24',
        'm36': 'm36',
        'm48': 'm48',
        'm60': 'm60',
        'm72': 'm72',
        'm84': 'm84',
        'm96': 'm96',
        'm108': 'm108',
        'm120': 'm120'
    }
    
    # Check for exact matches first
    if viscode in visit_mappings:
        return visit_mappings[viscode]
    
    # Check for pattern matches (e.g., '4_init', '4_m12')
    if '_init' in viscode:
        return 'bl'  # Initial visits often correspond to baseline
    
    # Extract month patterns (e.g., 'm12', '12m', '12_month')
    month_match = re.search(r'(\d+)', viscode)
    if month_match:
        month_num = int(month_match.group(1))
        if month_num == 0:
            return 'bl'
        else:
            return f'm{month_num:02d}'
    
    return viscode

def match_proteomic_mri_data(csv1_path, csv2_path, output_path=None):
    """
    Match and join proteomic data with MRI data based on RID-Subject mapping and VISCODE-Visit matching
    
    Args:
        csv1_path (str): Path to proteomic data CSV (with RID, VISCODE)
        csv2_path (str): Path to MRI data CSV (with Subject, Visit)
        output_path (str): Path to save merged CSV (optional)
        
    Returns:
        pd.DataFrame: Merged dataset
    """
    print("🔍 Loading datasets...")
    print("=" * 60)
    
    # Load CSV files
    print(f"📂 Loading proteomic data: {csv1_path}")
    df_proteomic = pd.read_csv(csv1_path)
    print(f"📊 Proteomic data: {len(df_proteomic)} records")
    
    print(f"📂 Loading MRI data: {csv2_path}")
    df_mri = pd.read_csv(csv2_path)
    print(f"📊 MRI data: {len(df_mri)} records")
    
    # Display sample data
    print("\n🔍 Sample proteomic data:")
    print(df_proteomic[['RID', 'VISCODE', 'research_group', 'subject_age']].head())
    
    print("\n🔍 Sample MRI data:")
    print(df_mri[['Subject', 'Visit', 'Group', 'Age', 'Description']].head())
    
    # Extract RID from Subject ID in MRI data
    print("\n🔄 Processing Subject IDs to extract RID...")
    df_mri['RID_extracted'] = df_mri['Subject'].apply(extract_rid_from_subject)
    
    # Normalize visit codes
    print("🔄 Normalizing visit codes...")
    df_proteomic['VISCODE_norm'] = df_proteomic['VISCODE'].apply(normalize_viscode)
    df_mri['Visit_norm'] = df_mri['Visit'].apply(normalize_viscode)
    
    # Display mapping examples
    print("\n📋 RID Extraction Examples:")
    sample_subjects = df_mri[['Subject', 'RID_extracted']].drop_duplicates().head(10)
    for _, row in sample_subjects.iterrows():
        print(f"  {row['Subject']} -> RID: {row['RID_extracted']}")
    
    print("\n📋 Visit Code Normalization Examples:")
    print("Proteomic VISCODE:")
    viscode_examples = df_proteomic[['VISCODE', 'VISCODE_norm']].drop_duplicates().head(5)
    for _, row in viscode_examples.iterrows():
        print(f"  {row['VISCODE']} -> {row['VISCODE_norm']}")
    
    print("MRI Visit:")
    visit_examples = df_mri[['Visit', 'Visit_norm']].drop_duplicates().head(5)
    for _, row in visit_examples.iterrows():
        print(f"  {row['Visit']} -> {row['Visit_norm']}")
    
    # Perform the merge
    print("\n🔗 Matching datasets on RID and Visit...")
    merged_df = pd.merge(
        df_proteomic,
        df_mri,
        left_on=['RID', 'VISCODE_norm'],
        right_on=['RID_extracted', 'Visit_norm'],
        how='inner',
        suffixes=('_proteomic', '_mri')
    )
    
    print(f"✅ Successfully matched {len(merged_df)} records")
    
    # Display merge statistics
    print("\n📊 MERGE STATISTICS")
    print("=" * 60)
    print(f"Original proteomic records: {len(df_proteomic)}")
    print(f"Original MRI records: {len(df_mri)}")
    print(f"Matched records: {len(merged_df)}")
    print(f"Proteomic match rate: {len(merged_df)/len(df_proteomic)*100:.1f}%")
    print(f"MRI match rate: {len(merged_df)/len(df_mri)*100:.1f}%")
    
    # Unique subjects and visits in merged data
    unique_subjects = merged_df['RID'].nunique()
    unique_visits = merged_df['VISCODE_norm'].nunique()
    print(f"Unique subjects in merged data: {unique_subjects}")
    print(f"Unique visits in merged data: {unique_visits}")
    
    # Group distribution
    if 'research_group' in merged_df.columns and 'Group' in merged_df.columns:
        print("\n📋 Group Distribution Comparison:")
        print("Proteomic groups:")
        proteomic_groups = merged_df['research_group'].value_counts()
        for group, count in proteomic_groups.items():
            print(f"  {group}: {count}")
        
        print("MRI groups:")
        mri_groups = merged_df['Group'].value_counts()
        for group, count in mri_groups.items():
            print(f"  {group}: {count}")
    
    # Age comparison
    if 'subject_age' in merged_df.columns and 'Age' in merged_df.columns:
        print("\n📊 Age Statistics:")
        age_diff = merged_df['Age'] - merged_df['subject_age']
        print(f"Age difference (MRI - Proteomic):")
        print(f"  Mean: {age_diff.mean():.2f}")
        print(f"  Std: {age_diff.std():.2f}")
        print(f"  Range: {age_diff.min():.2f} to {age_diff.max():.2f}")
    
    # Sample of merged data
    print("\n🔍 Sample merged data:")
    sample_cols = ['RID', 'VISCODE_norm', 'Subject', 'Visit_norm', 'research_group', 'Group', 'Description']
    available_cols = [col for col in sample_cols if col in merged_df.columns]
    print(merged_df[available_cols].head())
    
    # Save merged data
    if output_path:
        print(f"\n💾 Saving merged data to: {output_path}")
        merged_df.to_csv(output_path, index=False)
        print("✅ Merged data saved successfully!")
    
    return merged_df

def analyze_unmatched_data(csv1_path, csv2_path):
    """
    Analyze records that didn't match between the two datasets
    
    Args:
        csv1_path (str): Path to proteomic data CSV
        csv2_path (str): Path to MRI data CSV
    """
    print("\n🔍 ANALYZING UNMATCHED DATA")
    print("=" * 60)
    
    # Load data
    df_proteomic = pd.read_csv(csv1_path)
    df_mri = pd.read_csv(csv2_path)
    
    # Process data
    df_mri['RID_extracted'] = df_mri['Subject'].apply(extract_rid_from_subject)
    df_proteomic['VISCODE_norm'] = df_proteomic['VISCODE'].apply(normalize_viscode)
    df_mri['Visit_norm'] = df_mri['Visit'].apply(normalize_viscode)
    
    # Find unmatched proteomic records
    proteomic_keys = set(zip(df_proteomic['RID'], df_proteomic['VISCODE_norm']))
    mri_keys = set(zip(df_mri['RID_extracted'], df_mri['Visit_norm']))
    
    unmatched_proteomic = proteomic_keys - mri_keys
    unmatched_mri = mri_keys - proteomic_keys
    
    print(f"Unmatched proteomic records: {len(unmatched_proteomic)}")
    print(f"Unmatched MRI records: {len(unmatched_mri)}")
    
    # Show some examples of unmatched proteomic records
    if unmatched_proteomic:
        print("\n📋 Sample unmatched proteomic records:")
        sample_unmatched = list(unmatched_proteomic)[:10]
        for rid, visit in sample_unmatched:
            print(f"  RID: {rid}, Visit: {visit}")
    
    # Show unique visits in each dataset
    print(f"\nUnique visits in proteomic data: {sorted(df_proteomic['VISCODE_norm'].unique())}")
    print(f"Unique visits in MRI data: {sorted(df_mri['Visit_norm'].unique())}")
    
    # Show unique RIDs that don't match
    proteomic_rids = set(df_proteomic['RID'].dropna())
    mri_rids = set(df_mri['RID_extracted'].dropna())
    
    rids_only_proteomic = proteomic_rids - mri_rids
    rids_only_mri = mri_rids - proteomic_rids
    
    print(f"\nRIDs only in proteomic data: {len(rids_only_proteomic)}")
    if rids_only_proteomic:
        print(f"  Examples: {sorted(list(rids_only_proteomic))[:10]}")
    
    print(f"RIDs only in MRI data: {len(rids_only_mri)}")
    if rids_only_mri:
        print(f"  Examples: {sorted(list(rids_only_mri))[:10]}")

if __name__ == "__main__":
    # File paths
    csv1_path = r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\proteomic_mri_with_labels.csv"
    csv2_path = r"C:\Users\User\github_repos\AD_CN_all_available_data_final\AD_CN_all_available_data.csv"
    output_path = r"D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\merged_proteomic_mri_data.csv"
    
    # Match the datasets
    merged_data = match_proteomic_mri_data(csv1_path, csv2_path, output_path)
    
    # Analyze unmatched data
    analyze_unmatched_data(csv1_path, csv2_path)
    
    print("\n✅ Data matching completed!")
    print(f"📊 Final merged dataset: {len(merged_data)} records")
    print(f"💾 Saved to: {output_path}")


🔍 Loading datasets...
📂 Loading proteomic data: D:\ADNI\AD_CN\proteomics\Biomarkers Consortium Plasma Proteomics MRM\proteomic_mri_with_labels.csv
📊 Proteomic data: 39 records
📂 Loading MRI data: C:\Users\User\github_repos\AD_CN_all_available_data_final\AD_CN_all_available_data.csv
📊 MRI data: 31182 records

🔍 Sample proteomic data:
   RID VISCODE research_group  subject_age
0   29      bl             AD        64.16
1   31      bl             CN        77.75
2   61      bl             CN        77.12
3   86      bl             CN        80.32
4  139      bl             AD        65.86

🔍 Sample MRI data:
      Subject   Visit Group  Age                          Description
0  941_S_7087      sc    CN   67          Accelerated Sagittal MPRAGE
1  941_S_7087      sc    CN   67                        Field Mapping
2  941_S_7074      sc    CN   71          Accelerated Sagittal MPRAGE
3  941_S_7074      sc    CN   71                        Field Mapping
4  941_S_7074  4_init    CN   72  Acc